In [1]:
import pandas as pd
from rdkit import rdBase
from rdkit import Chem
from rdkit.Chem.rdmolfiles import SmilesWriter
from rdkit.Chem.rdmolfiles import SDWriter

zinc = pd.read_csv('../../big datasets/zinc_with_fps.csv', low_memory=False)

descriptors = pd.read_csv('../../big datasets/csv/concat csv/zinc_descriptors.csv')

zinc.head()
zinc1=zinc[['ZINC_ID', 'MWT', 'SMILES']]

dd1 = descriptors.drop(['Name'], axis=1)

zincall = zinc1.join(dd1, how='left')

zincall.fillna(0, inplace=True)

zincall.isnull().sum().sum()

zincall.to_csv('../../big datasets/featurized_zinc.csv')

zincall.iloc[:,3:1448].astype(float)

In [5]:
from sklearn.utils import shuffle
from sklearn.feature_selection import VarianceThreshold 


In [2]:
zincall = pd.read_csv('../../big datasets/featurized_zinc.csv')

In [13]:
half = zincall.sample(frac=0.5)
half.head()
hal = half.iloc[:, 6:]
hal.head()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
96,0.0,0.0000,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000,0.0
18542,0.0,0.2282,0.052075,127.3531,66.375825,0.0,0.0,56.0,31.0,25.0,...,7.414100,64.582529,2.083307,18.406615,8.158495,10.248120,2309.0,59.0,1.466,172.0
42407,0.0,1.8844,3.550963,93.0641,44.507895,0.0,0.0,36.0,21.0,15.0,...,8.390860,42.017990,2.000857,10.672885,8.154633,0.000000,1020.0,30.0,2.540,102.0
11721,0.0,-1.2199,1.488156,104.2819,57.271032,0.0,0.0,50.0,26.0,24.0,...,7.083411,54.626051,2.101002,15.567999,11.821640,3.746359,1516.0,51.0,1.382,152.0
40171,0.0,0.4304,0.185244,92.4567,43.660516,0.0,0.0,34.0,22.0,12.0,...,9.237124,44.793114,2.036051,13.723590,2.618750,6.034429,942.0,38.0,0.190,116.0


In [8]:
constant_filter = VarianceThreshold(threshold=0)
qconstant_filter = VarianceThreshold(threshold=0.01)

In [14]:
constant_filter.fit(hal)
len(hal.columns[constant_filter.get_support()])
constant_columns = [column for column in hal.columns if column
                   not in hal.columns[constant_filter.get_support()]]

hal.drop(labels=constant_columns, axis=1, inplace=True)
#test_x.drop(labels=constant_columns, axis=1, inplace=True)

In [15]:
qconstant_filter.fit(hal)
len(hal.columns[qconstant_filter.get_support()])

q_constant_columns = [column for column in hal.columns if column
                     not in hal.columns[qconstant_filter.get_support()]]

half = qconstant_filter.transform(hal)
#test_x = qconstant_filter.transform(test_x)

hal.shape #, test_x.shape

(50872, 1210)

In [16]:
correlated_features = set()
correlation_matrix = pd.DataFrame(hal).corr()

In [17]:
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i,j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [18]:
len(correlated_features)

915

In [19]:
hal = pd.DataFrame(hal)
#test_x = pd.DataFrame(test_x)

In [20]:
hal.drop(labels=correlated_features, axis=1, inplace=True)
#test_x.drop(labels=correlated_features, axis=1, inplace=True)

In [27]:
hal.shape
use = hal.sample(frac=0.02)
use.shape
use.to_csv('../../big datasets/use_me_zinc_positive.csv')

df1 = df.head(5)

df1

In [3]:
from rdkit.Chem.Fingerprints import FingerprintMols
def fingerprint(input_df):
    '''From the input dataframe, makes a list of rdkit Mol objects and makes a
    list of rdkit fingerprints generated from those Mol objects. Inserts both
    lists as new columns and returns the expanded dataframe.'''
    
    mol_list = []
    fp_list = []

    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list

    input_df['Mol'] = mol_list
    input_df['Fingerprint'] = fp_list

    return input_df

In [5]:
fingerprint(zinc4)

C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Unnamed: 0,ZINC_ID,MWT,LogP,Desolv_apolar,Desolv_polar,HBD,HBA,tPSA,Charge,NRB,SMILES,Mol,Fingerprint
74001,74001,ZINC15957624,495.535,3.91,11.26,-18.52,2,8,105,0,6,CCOC(=O)c1ccc(cc1)N2C(=O)[C@@H]3[C@@H](N[C@]4(...,<rdkit.Chem.rdchem.Mol object at 0x0000020219B...,"[0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74002,74002,ZINC15957629,424.477,2.06,7.78,-62.49,2,8,119,-1,4,c1ccc2c(c1)[C@]3([C@H]4[C@@H]([C@@H](N3)CCC(=O...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74003,74003,ZINC15957630,424.477,2.06,6.45,-54.54,2,8,119,-1,4,c1ccc2c(c1)[C@]3([C@H]4[C@@H]([C@H](N3)CCC(=O)...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74004,74004,ZINC15957631,424.477,2.06,6.28,-55.23,2,8,119,-1,4,c1ccc2c(c1)[C@@]3([C@H]4[C@@H]([C@@H](N3)CCC(=...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74005,74005,ZINC15957632,424.477,2.06,6.27,-50.24,2,8,119,-1,4,c1ccc2c(c1)[C@@]3([C@H]4[C@@H]([C@H](N3)CCC(=O...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74006,74006,ZINC15957653,468.897,2.15,3.13,-19.33,4,9,131,0,5,COc1ccccc1CN2C(=O)[C@@H]3[C@H](N[C@]4([C@@H]3C...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74007,74007,ZINC15957654,468.897,2.15,4.37,-15.99,4,9,131,0,5,COc1ccccc1CN2C(=O)[C@@H]3[C@@H](N[C@]4([C@@H]3...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
74008,74008,ZINC15957666,424.501,2.80,2.52,-15.52,4,8,122,0,3,Cc1ccc2c(c1C)NC(=O)[C@]23[C@H]4[C@@H]([C@H](N3...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
74009,74009,ZINC15957667,424.501,2.80,2.65,-24.61,4,8,122,0,3,Cc1ccc2c(c1C)NC(=O)[C@@]23[C@H]4[C@@H]([C@H](N...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
74010,74010,ZINC15957668,494.979,4.68,5.54,-22.43,4,8,122,0,6,CCCCc1ccc(cc1)N2C(=O)[C@@H]3[C@H](N[C@]4([C@@H...,<rdkit.Chem.rdchem.Mol object at 0x000002021A7...,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
mols = [mol for mol in zinc4['Mol']]
writer = SDWriter('zinc_smiles4.sdf')
for mol in mols:
    writer.write(mol)
writer.close()

$ java -jar PaDEL-Descriptor.jar -retainorder -dir molecules -file biodeg_descriptors1.csv -2d -maxruntime 2000

df.shape

In [ ]:
java -jar PaDEL-Descriptor.jar -retainorder -dir molecules -file zinc.csv -2d -maxruntime 2000 -threads -1 -waitingjo
bs -1 -maxcpdperfile 2000

In [ ]:
plan: split the master zinc list into chunks: one 45,000, one the remainder

In [12]:
34001+45000

79001

In [13]:
zinc2 = zinc.iloc[34001:79001,:]

In [14]:
zinc2.shape

(45000, 14)

In [15]:
zinc3 = zinc.iloc[79001:,:]

In [16]:
zinc3.shape

(22744, 14)

In [4]:
zinc4 = zinc.iloc[74001:79001,:]

In [ ]:
zinc2 = 34001:74001

In [ ]:
zinc 3 = 79001:end

In [ ]:
zinc 4 = 74001:79001

In [4]:
zinc.head()

,Unnamed: 0,ZINC_ID,MWT,LogP,Desolv_apolar,Desolv_polar,HBD,HBA,tPSA,Charge,NRB,SMILES,Mol,Fingerprint
0,0,ZINC00000010,217.200,1.42,5.57,-41.98,0,4,66,-1,2,C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2,<rdkit.Chem.rdchem.Mol object at 0x000001708A9...,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...
1,1,ZINC00000012,289.356,1.28,4.89,-24.55,2,4,66,0,5,c1ccc(cc1)C(c2ccccc2)[S@](=O)CC(=O)NO,<rdkit.Chem.rdchem.Mol object at 0x000001708DE...,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...
2,2,ZINC00000017,281.337,1.33,3.06,-23.33,2,6,87,0,4,CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N\C(=O)OC)/[nH]2,<rdkit.Chem.rdchem.Mol object at 0x000001708A9...,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...
3,3,ZINC00000017,281.337,1.33,3.07,-19.20,2,6,87,0,4,CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N/C(=O)OC)/[nH]2,<rdkit.Chem.rdchem.Mol object at 0x000001708A9...,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...
4,4,ZINC00000022,218.276,3.21,0.47,-48.57,1,3,52,-1,5,C[C@@H](c1ccc(cc1)NCC(=C)C)C(=O)[O-],<rdkit.Chem.rdchem.Mol object at 0x000001708A9...,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...


In [6]:
zinc1=zinc[['SMILES', 'ZINC_ID']]

In [7]:
zinc1.head()

,SMILES,ZINC_ID
0,C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2,ZINC00000010
1,c1ccc(cc1)C(c2ccccc2)[S@](=O)CC(=O)NO,ZINC00000012
2,CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N\C(=O)OC)/[nH]2,ZINC00000017
3,CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N/C(=O)OC)/[nH]2,ZINC00000017
4,C[C@@H](c1ccc(cc1)NCC(=C)C)C(=O)[O-],ZINC00000022


In [55]:
deg2 = deg1.astype(str)

In [3]:
dd = pd.read_csv('../../Downloads/PaDEL-Descriptor/biodeg_descriptors.csv', low_memory=False)

In [4]:
dd.head()

,Name,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,AUTOGEN_biodeg_smiles_1,0.0,-2.4894,6.197112,38.0018,35.363032,0.0,0,35,11,...,4.462509,20.828762,1.893524,0.000000,0.000000,0.0,220,8,6.599,38
1,AUTOGEN_biodeg_smiles_2,0.0,-0.4296,0.184556,18.5395,13.176344,0.0,0,13,5,...,5.542886,9.148995,1.829799,2.705816,2.705816,0.0,17,2,0.628,22
2,AUTOGEN_biodeg_smiles_3,0.0,-0.5003,0.250300,45.6856,29.290274,0.0,0,30,12,...,5.937350,22.828595,1.902383,11.163248,11.163248,0.0,286,9,-0.646,42
3,AUTOGEN_biodeg_smiles_4,1.0,-0.3087,0.095296,8.5717,4.697586,0.0,0,5,3,...,9.201096,4.914214,1.638071,4.414214,4.414214,0.0,4,0,-0.316,6
4,AUTOGEN_biodeg_smiles_5,0.0,-0.8561,0.732907,50.2372,34.143860,0.0,0,34,14,...,6.004005,26.661622,1.904402,11.202922,11.202922,0.0,444,11,1.211,52


In [6]:
dd1 = dd.drop(['Name'], axis=1)

In [79]:
dd1.head()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,0,-2.4894,6.19711236,38.0018,35.363032,0,0,35,11,24,...,4.462508593,20.82876223,1.893523839,0,0,0,220,8,6.599,38
1,0,-0.4296,0.18455616,18.5395,13.176344,0,0,13,5,8,...,5.54288576,9.148995135,1.829799027,2.705815798,2.705815798,0,17,2,0.628,22
2,0,-0.5003,0.25030009,45.6856,29.290274,0,0,30,12,18,...,5.937350302,22.82859468,1.90238289,11.16324806,11.16324806,0,286,9,-0.646,42
3,1,-0.3087,0.09529569,8.5717,4.697586,0,0,5,3,2,...,9.201095861,4.914213562,1.638071187,4.414213562,4.414213562,0,4,0,-0.316,6
4,0,-0.8561,0.73290721,50.2372,34.14386,0,0,34,14,20,...,6.00400468,26.6616219,1.904401564,11.20292211,11.20292211,0,444,11,1.211,52


In [120]:
dd2=dd1.astype(float)

ValueError: could not convert string to float: '#NAME?'

In [11]:
biodeg = deg1.join(dd1, how='left')

In [12]:
biodeg.shape

(1055, 1447)

In [17]:
biodeg.head()

,SMILES,Class,Status,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,CCCCCCCCCCC,RB,Train,0.0,-2.4894,6.197112,38.0018,35.363032,0.0,0,...,4.462509,20.828762,1.893524,0.000000,0.000000,0.0,220,8,6.599,38
1,CCC1CO1,RB,Train,0.0,-0.4296,0.184556,18.5395,13.176344,0.0,0,...,5.542886,9.148995,1.829799,2.705816,2.705816,0.0,17,2,0.628,22
2,CCOCCOCCOCCO,RB,Train,0.0,-0.5003,0.250300,45.6856,29.290274,0.0,0,...,5.937350,22.828595,1.902383,11.163248,11.163248,0.0,286,9,-0.646,42
3,OC=O,RB,Train,1.0,-0.3087,0.095296,8.5717,4.697586,0.0,0,...,9.201096,4.914214,1.638071,4.414214,4.414214,0.0,4,0,-0.316,6
4,CCCCOCCOCCOC(C)=O,RB,Train,0.0,-0.8561,0.732907,50.2372,34.143860,0.0,0,...,6.004005,26.661622,1.904402,11.202922,11.202922,0.0,444,11,1.211,52


In [19]:
biodeg.isnull().sum().sum()

2661

In [20]:
biodeg1 = biodeg.fillna(0.0)

In [22]:
biodeg1.isnull().sum().sum()

0

In [23]:
biodeg1.to_csv('master_biodeg_descriptors.csv')